# Technical Analysis of Stocks in Python



This notebook serves as a exploration of technical analysis techniques applied to stock market data. We will delve into various technical indicators including moving averages, Relative Strength Index (RSI), Bollinger Bands, and Average True Range (ATR).

The primary objective of this notebook is to visualize and analyze the behavior of stock prices, identify potential trends, and gain insights into market sentiment.

This notebook will consist of the following components:

- Basic Candlestick Chart: A foundation for our analysis, providing a visual representation of price action over time.

- Moving Averages: Calculation and visualization of simple and exponential moving averages to identify trends and support/resistance levels.

- Relative Strength Index (RSI): Assessment of overbought and oversold conditions to gauge momentum and potential reversal points.

- Bollinger Bands: Identification of price volatility and potential mean reversion opportunities.

- Average True Range (ATR): Measurement of price volatility to assist in risk management and stop-loss placement.

- Combined Technical Indicator Chart: A comprehensive visualization incorporating all the aforementioned indicators for a holistic analysis.


In [ ]:
#install dependencies
!pip install yfinance
!pip install quantstats
!pip install pyportfolioopt
!pip install ta
!pip install plotly --upgrade

In [ ]:
# Importing Libraries

# Data handling and statistical analysis
import pandas as pd
from pandas_datareader import data
import numpy as np
from scipy import stats

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from matplotlib import rcParams

# Financial data
import quantstats as qs
import ta
import yfinance as yf

# Enabling Plotly offline
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Datetime and hiding warnings
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import logging

# Optimization and allocation
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import black_litterman, BlackLittermanModel

Let's start with Apple stock and let's plot a 10-year candlestick chart with volume and without any indicators.

In [ ]:
#downloading apple stock data
aapl = yf.download('AAPL', start = '2014-07-01', end = '2024-06-15')

[*********************100%***********************]  1 of 1 completed


In [ ]:
#setting plotly to render for colab
pio.renderers.default = 'colab'

In [ ]:
#plotting candlestick chart without indicators

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights = [0.7, 0.3])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)


#volume chart
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=2, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 2024',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

fig.show()

## Moving Averages

Traders across various markets, including stocks, commodities, forex, and crypto, rely on moving averages to navigate the often-choppy waters of price movements. These averages act like a smoothing filter, reducing short-term fluctuations and revealing the underlying trend.

Think of it like this: a moving average is simply the average price of an asset calculated over a specific timeframe. This timeframe can be anything from a few days (like 9) to a much longer period (like 200 days), depending on the trader's goals and the asset they're analyzing. The resulting line, plotted on a chart, becomes a visual guide for gauging the trend's direction and strength.

There are two main types of moving averages: simple moving averages (SMA) and exponential moving averages (EMA).

**Simple Moving Average (SMA)** : This is the workhorse. It adds up the closing prices for a chosen period and divides by the number of days. For instance, a 20-day SMA simply averages the closing prices of the past 20 days.

**Exponential Moving Average (EMA)** : This one pays closer attention to what's happening recently. It assigns more weight to newer price data, making it more responsive to recent price movements.

Traders leverage moving averages for various purposes:


*   *Identifying Support and Resistance* : Moving averages can act as dynamic support
and resistance levels. When the price is above the moving average, it often suggests an uptrend. Conversely, a price below the line hints at a downtrend.

*   *Spotting Trend Changes* : Watch for "crossovers" between different moving averages. For example, the Golden Cross (50-day SMA crossing above the 200-day SMA) can signal a potential shift towards an uptrend, while the Death Cross (50-day SMA crossing below the 200-day SMA) might indicate a downtrend.

*   *Managing Trades* : Moving averages can help set stop-loss and take-profit levels. A stop-loss placed below the moving average can limit potential losses if the trend reverses.

In [ ]:
#adding Moving Averages

#exponential 9-Period Moving Average
aapl['EMA9'] = aapl['Adj Close'].ewm(span = 9, adjust = False).mean()

#simple 20-Period Moving Average
aapl['SMA20'] = aapl['Adj Close'].rolling(window=20).mean()

#simple 50-Period Moving Average
aapl['SMA50'] = aapl['Adj Close'].rolling(window=50).mean()

#simple 100-Period Moving Average
aapl['SMA100'] = aapl['Adj Close'].rolling(window=100).mean()

#simple 200-Period Moving Average
aapl['SMA200'] = aapl['Adj Close'].rolling(window=200).mean()

In [ ]:
#plotting candlestick chart with Moving Averages

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights = [0.7, 0.3])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#moving Averages
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['EMA9'],
                         mode='lines',
                         line=dict(color='#90EE90'),
                         name='EMA9'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA20'],
                         mode='lines',
                         line=dict(color='yellow'),
                         name='SMA20'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA50'],
                         mode='lines',
                         line=dict(color='orange'),
                         name='SMA50'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA100'],
                         mode='lines',
                         line=dict(color='purple'),
                         name='SMA100'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA200'],
                         mode='lines',
                         line=dict(color='red'),
                         name='SMA200'),
              row=1, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=2, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 2024',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#configuring axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

fig.show()

## RSI

The Relative Strength Index (RSI) is a well-regarded technical analysis tool that helps traders assess the strength of price movements. By comparing recent gains and losses, it can provide valuable insights into potential buy and sell opportunities.

Here's how it works:



1.   *Calculate Gains and Losses* : For each day, the difference between the current closing price and the previous day's closing price is calculated. Positive differences are gains, while negative ones are losses.

2.   *Average Gains and Losses* : The average of these gains and losses is computed.

3.   *Calculate RS* : The RS is the ratio of the average gains to the average losses.


The RSI value is then plotted on a chart. A reading above 70 generally indicates an overbought condition, suggesting potential selling opportunities. Conversely, a reading below 30 often signals an oversold condition, hinting at potential buying opportunities.

Traders also watch for divergences between the RSI and the price. If the price is making new highs, but the RSI is not, it could suggest a loss of momentum and a potential reversal.

In [ ]:
#adding RSI for 14-periods

delta = aapl['Adj Close'].diff()
gain = delta.where(delta > 0,0)
loss = -delta.where(delta < 0,0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain/avg_loss

#adding RSI column to the df
aapl['RSI'] = 100 - (100 / (1 + rs))

In [ ]:
#plotting candlestick chart with RSI

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.5, 0.25, 0.25])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#relative Strengh Index (RSI)
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['RSI'],
                         mode='lines',
                         line=dict(color='#CBC3E3'),
                         name='RSI'),
              row=2, col=1)

#adding marking lines at 70 and 30 levels
fig.add_shape(type="line",
              x0=aapl.index[0], y0=70, x1=aapl.index[-1], y1=70,
              line=dict(color="red", width=2, dash="dot"),
              row=2, col=1)
fig.add_shape(type="line",
              x0=aapl.index[0], y0=30, x1=aapl.index[-1], y1=30,
              line=dict(color="#90EE90", width=2, dash="dot"),
              row=2, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=3, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 2024',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#configuring axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='RSI', row=2, col=1)
fig.update_yaxes(title_text='Volume', row=3, col=1)

fig.show()

## Bollinger Bands

Bollinger Bands are a widely used technical analysis tool that provides insights into market volatility and helps traders identify potential entry points.

There are three lines on the chart: a middle line representing a simple moving average (SMA), typically a 20-day average, and two outer bands positioned above and below the SMA. These bands are set at a distance of two standard deviations from the SMA.

When the market is volatile, the bands expand, indicating increased price fluctuations. Conversely, when the market is calm, the bands contract, suggesting lower volatility. The distance between the bands serves as a visual indicator of volatility.

Traders often use Bollinger Bands to spot potential buying and selling opportunities:


*   **Oversold** : When the price touches or breaks below the lower band, it might signal an oversold condition, suggesting a possible buying opportunity.

*   **Overbought** : Conversely, a price touching or breaking above the upper band could indicate an overbought condition, suggesting a potential selling opportunity

To calculate Bollinger Bands, we follow these steps:


1.   *Calculate the SMA* : Determine the 20-day simple moving average of the stock's price.

2.   *Calculate Standard Deviation* : Calculate the standard deviation of the stock's price over the past 20 periods.

3.   *Determine Bands* : Add and subtract two standard deviations from the SMA to obtain the upper and lower bands.

In [ ]:
#adding Bollinger Bands 20-periods

aapl['BB_UPPER'] = aapl['SMA20'] + 2*aapl['Adj Close'].rolling(window=20).std()
aapl['BB_LOWER'] = aapl['SMA20'] - 2*aapl['Adj Close'].rolling(window=20).std()

In [ ]:
#plotting candlestick chart with bollinger bands

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights = [0.7, 0.3])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#bollinger Bands
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_UPPER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Upper Band'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_LOWER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Lower Band'),
              row=1, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=2, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 2024',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#configuring axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

fig.show()

## ATR

The Average True Range (ATR) is a popular indicator used to gauge the volatility of price movements. It helps traders assess the potential for significant price swings and identify potential trading opportunities.

To calculate the ATR, traders compare the "true range" of a stock over a specific period. The true range is the largest of these three differences:


1.   **High minus Previous Close** : The difference between the current high price and the previous day's closing price.

2.   **Low minus Previous Close** : The difference between the current low price and the previous day's closing price.

3.   **High minus Low** : The difference between the current day's high and low prices.

The ATR ranges from 0 to infinity. A higher ATR value suggests increased volatility, while a lower value indicates lower volatility.

Traders often use the ATR to:


*   *Set Stop-Loss and Take-Profit Levels* : By considering the ATR, traders can set more informed stop-loss and take-profit levels.

*   *Identify Trend Reversals* : An increasing ATR might signal a potential trend reversal, as it indicates growing volatility.

In [ ]:
#adding ATR 14-periods

aapl['TR'] = pd.DataFrame(np.maximum(np.maximum(aapl['High'] - aapl['Low'], abs(aapl['High'] - aapl['Adj Close'].shift())), abs(aapl['Low'] - aapl['Adj Close'].shift())), index = aapl.index)

#adding ATR to the df
aapl['ATR'] = aapl['TR'].rolling(window = 14).mean()

In [ ]:
# Plotting candlestick chart with ATR


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.5, 0.25, 0.25])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#Average True Range (ATR)
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['ATR'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='ATR'),
              row=2, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=3, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 2024',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#configuring axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='ATR', row=2, col=1)
fig.update_yaxes(title_text='Volume', row=3, col=1)

fig.show()

## All Technical Indicators

In this section, we will add all of the technical indicators above to the same chart.

In [ ]:
#plotting Candlestick charts with all indicators

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,row_heights=[0.6, 0.10, 0.10, 0.20])

#candlestick
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#Moving Averages
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['EMA9'],
                         mode='lines',
                         line=dict(color='#90EE90'),
                         name='EMA9'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA20'],
                         mode='lines',
                         line=dict(color='yellow'),
                         name='SMA20'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA50'],
                         mode='lines',
                         line=dict(color='orange'),
                         name='SMA50'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA100'],
                         mode='lines',
                         line=dict(color='purple'),
                         name='SMA100'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA200'],
                         mode='lines',
                         line=dict(color='red'),
                         name='SMA200'),
              row=1, col=1)

#Bollinger Bands
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_UPPER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Upper Band'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_LOWER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Lower Band'),
              row=1, col=1)

fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#Relative Strengh Index (RSI)
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['RSI'],
                         mode='lines',
                         line=dict(color='#CBC3E3'),
                         name='RSI'),
              row=2, col=1)

#adding marking lines at 70 and 30 levels
fig.add_shape(type="line",
              x0=aapl.index[0], y0=70, x1=aapl.index[-1], y1=70,
              line=dict(color="red", width=2, dash="dot"),
              row=2, col=1)
fig.add_shape(type="line",
              x0=aapl.index[0], y0=30, x1=aapl.index[-1], y1=30,
              line=dict(color="#90EE90", width=2, dash="dot"),
              row=2, col=1)

#Average True Range (ATR)
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['ATR'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='ATR'),
              row=3, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=4, col=1)

#layout
fig.update_layout(title='AAPL Candlestick Chart From July 1st, 2014 to June 15th, 202',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=4, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='RSI', row=2, col=1)
fig.update_yaxes(title_text='ATR', row=3, col=1)
fig.update_yaxes(title_text='Volume', row=4, col=1)

fig.show()

## Pick and Choose

We can pick and choose which indicators to add. Here, I will be adding SMA20 as well Bollinger Bands.

In [ ]:
#downloading Apple stocks
aapl = yf.download('AAPL', start = '2024-01-15', end = '2024-06-15')

[*********************100%***********************]  1 of 1 completed


In [ ]:
#Moving Averages

#simple 20-Period Moving Average
aapl['SMA20'] = aapl['Adj Close'].rolling(window=20).mean()

In [ ]:
#adding Bollinger Bands 20-periods
aapl['BB_UPPER'] = aapl['SMA20'] + 2*aapl['Adj Close'].rolling(window=20).std() # Upper Band
aapl['BB_LOWER'] = aapl['SMA20'] - 2*aapl['Adj Close'].rolling(window=20).std() # Lower Band

In [ ]:
# Plotting candlestick chart with select indicators

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights = [0.7, 0.3])
fig.add_trace(go.Candlestick(x=aapl.index,
                             open=aapl['Open'],
                             high=aapl['High'],
                             low=aapl['Low'],
                             close=aapl['Adj Close'],
                             name='AAPL'),
              row=1, col=1)

#Moving Averages
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['SMA20'],
                         mode='lines',
                         line=dict(color='#90EE90'),
                         name='SMA20'),
              row=1, col=1)

#Bollinger Bands
fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_UPPER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Upper Band'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=aapl.index,
                         y=aapl['BB_LOWER'],
                         mode='lines',
                         line=dict(color='#00BFFF'),
                         name='Lower Band'),
              row=1, col=1)

#volume
fig.add_trace(go.Bar(x=aapl.index,
                     y=aapl['Volume'],
                     name='Volume',
                     marker=dict(color='orange', opacity=1.0)),
              row=2, col=1)

#annotation
fig.add_annotation(text='Apple (AAPL)',
                    font=dict(color='white', size=40),
                    xref='paper', yref='paper',
                    x=0.5, y=0.65,
                    showarrow=False,
                    opacity=0.2)

#layout
fig.update_layout(title='AAPL Candlestick Chart From January 15th, 2024 to June 15th, 202',
                  yaxis=dict(title='Price (USD)'),
                  height=1000,
                 template = 'plotly_dark')

#configuring axes
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=False, row=2, col=1)
fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

fig.show()